In [1]:
import numpy as np
import pandas as pd
import os
from google.colab import files
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
!pip install gdown
!pip install --upgrade gdown
!pip install patool
import patoolib

os.mkdir('unzipped_folder')

!gdown "https://drive.google.com/u/0/uc?id=1mipJf49ZvqD3CArtutXNOJriqdtb9tAH&export=download"

patoolib.extract_archive("/content/dataset.zip", outdir="/content/unzipped_folder")

ratings = pd.read_csv("/content/unzipped_folder/dataset/rating.csv")
animes = pd.read_csv("/content/unzipped_folder/dataset/anime.csv")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 3.3 MB/s eta 0:00:00
Downloading...
From (uriginal): https://drive.google.com/u/0/uc?id=1mipJf49ZvqD3CArtutXNOJriqdtb9tAH&export=download
From (redirected): https://drive.google.com/uc?id=1mipJf49ZvqD3CArtutXNOJriqdtb9tAH&export=download&confirm=t&uuid=adf58e69-93f4-43f0-b2bc-4f28fc3632c3
To: /content/dataset.zip
100% 26.3M/26.3M [00:00<00:00, 139MB/s]
patool: Extracting /content/dataset.zip ...
patool: running /usr/bin/7z x -o/content/unzipped_folder -- /content/dataset.zip
patool: ... /con

In [3]:
tfidf = TfidfVectorizer(stop_words='english')

N = len(animes['name'])
id_dic = dict(zip(list(range(N)), (animes["anime_id"])))

def clean_data(x):

    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''


features = ['name', 'genre', 'type', 'rating']

for feature in features:

    animes[feature] = animes[feature].apply(clean_data)


def create_soup(x):
  
    return ' ' + x['name'] + ' ' + x['genre'] + ' ' + x['type'] + ' ' + x['rating']


animes['soup'] = animes.apply(create_soup, axis=1)

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(animes['soup'])

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

indices = pd.Series(animes.index, index=animes['anime_id']).drop_duplicates()

def get_animes(user_id):

  select = ratings.loc[ratings['user_id'] == user_id]
  sel = select[['anime_id', 'rating']]
  sel_sorted = sel.sort_values(by = 'rating', ascending = False)
  all_animes = sel_sorted['anime_id'].tolist()
  top_animes = all_animes[0:5]

  return top_animes , all_animes
  

def get_recommendations(top_animes, all_animes, cosine_sim=cosine_sim2):

  rec_animes = []

  for i in top_animes:

    idx = indices[i]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]

    animes_indices = [i[0] for i in sim_scores]

    for j in animes_indices:

      anime_id = id_dic[j]
      
      if (anime_id not in rec_animes) and (anime_id not in all_animes):
        rec_animes.append(anime_id)

  return rec_animes


animes_2 = pd.read_csv("/content/unzipped_folder/dataset/anime.csv")
ids_names = dict(zip(animes_2['anime_id'], animes_2['name']))

In [4]:
user_id = int(input("Enter user id: "))

top_animes, all_animes = get_animes(user_id)

recommended_animes = get_recommendations(top_animes, all_animes, cosine_sim2)[0:10]

print("")
print("Recommended animes:")
print("")

for i in recommended_animes:

 print(ids_names[i])

Enter user id: 3

Recommended animes:

Diamond no Ace
Area no Kishi
Teekyuu 3
Mousou Dairinin
Higurashi no Naku Koro ni Kai
Higurashi no Naku Koro ni
Death Note Rewrite
Mirai Nikki (TV)
Pokemon: Pikachu Koori no Daibouken
Pokemon: Pikachu no Wanpaku Island
